In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
Crime_5_7 = pd.read_csv('C:\\Users\\kodur\\Documents\\Chicago_Crimes_2005_to_2007.csv.',
                        na_values = [None, 'NaN','Nothing'], header = 0) 
Crime_8_11 = pd.read_csv('C:\\Users\\kodur\\Documents\\Chicago_Crimes_2008_to_2011.csv.',
                        na_values = [None, 'NaN','Nothing'], header = 0) 
Crime_12_17 = pd.read_csv('C:\\Users\\kodur\\Documents\\Chicago_Crimes_2012_to_2017.csv.',
                        na_values = [None, 'NaN','Nothing'], header = 0)

In [3]:
ZipCode_Data = pd.read_csv('C:\\Users\\kodur\\Documents\\Chicago_Zip_Data.csv.', header = 0)

In [4]:
Crime_Data = [Crime_5_7, Crime_8_11, Crime_12_17]
del Crime_5_7
del Crime_8_11
del Crime_12_17
Crime_Data = pd.concat(Crime_Data,axis = 0)

In [5]:
Crime_Data.drop_duplicates(subset=['ID', 'Case Number'], inplace=True)

In [6]:
Crime_Data.drop(['Unnamed: 0','Case Number','IUCR','FBI Code','Location',
                 'X Coordinate','Y Coordinate','Location'], inplace = True, axis = 1)

In [7]:
Crime_Data.Date = pd.to_datetime(Crime_Data.Date, format = '%m/%d/%Y %I:%M:%S %p')
Crime_Data['Updated On'] = pd.to_datetime(Crime_Data['Updated On'], format = '%m/%d/%Y %I:%M:%S %p')
Crime_Data.index = pd.DatetimeIndex(Crime_Data.Date)

In [8]:
#will look into time it takes for an arrest later, so creating column for it
#turning other parts of data into categorical parts for later machine learning applications
Crime_Data['ArrestTime'] = Crime_Data['Updated On'] - Crime_Data.Date
Crime_Data['Primary Type'] = pd.Categorical(Crime_Data['Primary Type'])
Crime_Data['Description'] = pd.Categorical(Crime_Data['Description'])
Crime_Data['Location Description'] = pd.Categorical(Crime_Data['Location Description'])

In [9]:
Groups = Crime_Data.groupby(Crime_Data['Primary Type'])
Groups = dict(list(Groups))
Keys = list(Groups.keys())

In [10]:
#grouping sex crimes together to look into
SexCrimes_Data = [Groups['CRIM SEXUAL ASSAULT'], Groups['HUMAN TRAFFICKING'],
                  Groups['OFFENSE INVOLVING CHILDREN'], Groups['PROSTITUTION'],
                  Groups['SEX OFFENSE'], Groups['STALKING']]
SexCrimes_Data = pd.concat(SexCrimes_Data, axis = 0)
del Crime_Data
del Groups
del Keys

In [24]:
from geopy.geocoders import Nominatim

In [25]:
#remove NAs from Longitude and Latitude data
#create list of longitude and latitude data for geolocator to find zipcode data from
SexCrimes_Data = SexCrimes_Data.dropna(axis = 0, how = 'any')
Long = [None] * SexCrimes_Data.shape[0]
Lat = [None] * SexCrimes_Data.shape[0]
Lo = str(list(SexCrimes_Data.Longitude))
La = str(list(SexCrimes_Data.Latitude))

In [26]:
Long = Lo.split(",")
Lat = La.split(",")

In [27]:
#removing brackets around longitude and latitude data
Long[0] = Long[0][1:]
Long[-1] = Long[-1][:-1]
Lat[0] = Lat[0][1:]
Lat[-1] = Lat[-1][:-1]

In [28]:
#concatenating longitude and latitude data
Str = [None] * SexCrimes_Data.shape[0]
for i in range(len(Str)):
    Str[i] = Lat[i] + ", " + Long[i]